In [1]:
!pip install auto-gptq --quiet
!pip install --upgrade optimum --quiet
!pip install --upgrade accelerate --quiet

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# from transformers import AwqConfig, AutoConfig
from optimum.gptq import GPTQQuantizer, load_quantized_model

quantizer = GPTQQuantizer(bits=4, dataset="c4", block_name_to_quantize = "model.decoder.layers", model_seqlen = 2048)

model_name = "EleutherAI/pythia-70m-deduped"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
quantized_model = quantizer.quantize_model(model, tokenizer)
# TODO: save model

CUDA extension not installed.
CUDA extension not installed.
Found cached dataset json (/home/lev/.cache/huggingface/datasets/allenai___json/allenai--c4-6fbe877195f42de5/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


ValueError: empty range for randrange() (0, 0, 0)

In [ ]:
quantizer.save(model, "quantized")

In [ ]:
from huggingface_hub import hf_hub_download
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "Elriggs/pythia-70m-deduped"

# The layer around which we want to interpret by looking at the last layer
layer_focus = 0

In [ ]:
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import numpy as np
from transformer_lens import HookedTransformer
setting = "residual"
model_name = "EleutherAI/pythia-70m-deduped"

model = HookedTransformer.from_pretrained(model_name, device=device)

def get_cache_name_neurons(layer: int):
    if setting == "residual":
        cache_name = f"blocks.{layer}.hook_resid_post"
        neurons = model.cfg.d_model
    elif setting == "mlp":
        cache_name = f"blocks.{layer}.mlp.hook_post"
        neurons = model.cfg.d_mlp
    elif setting == "attention":
        cache_name = f"blocks.{layer}.hook_attn_out"
        neurons = model.cfg.d_model
    elif setting == "mlp_out":
        cache_name = f"blocks.{layer}.hook_mlp_out"
        neurons = model.cfg.d_model
    else:
        raise NotImplementedError
    return cache_name, neurons

cache_name, neurons  = get_cache_name_neurons(layer_focus)
cache_name_prior, _  = get_cache_name_neurons(layer_focus - 1)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m-deduped into HookedTransformer


In [ ]:
# Downnload dataset
from datasets import Dataset, load_dataset
dataset_name = "NeelNanda/pile-10k"
# dataset_name = "NeelNanda/pile-10k"
# dataset_name = "JeanKaddour/minipile"
token_amount= 60

dataset = load_dataset(dataset_name, split="train[:1000]").map(
    lambda x: model.tokenizer(x['text']),
    batched=True,
).filter(
    lambda x: len(x['input_ids']) > token_amount
).map(
    lambda x: {'input_ids': x['input_ids'][:token_amount]}
)
# TODO: we can maybe make this faster for the larger dataset?

Found cached dataset parquet (/home/lev/.cache/huggingface/datasets/NeelNanda___parquet/NeelNanda--pile-10k-72f566e9f7c464ab/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /home/lev/.cache/huggingface/datasets/NeelNanda___parquet/NeelNanda--pile-10k-72f566e9f7c464ab/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-3d37cf93bf4eabea.arrow
Loading cached processed dataset at /home/lev/.cache/huggingface/datasets/NeelNanda___parquet/NeelNanda--pile-10k-72f566e9f7c464ab/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-fb7554683ae70ef0.arrow
Loading cached processed dataset at /home/lev/.cache/huggingface/datasets/NeelNanda___parquet/NeelNanda--pile-10k-72f566e9f7c464ab/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-980bfa4a4898f16b.arrow


In [ ]:
from torch.utils.data import DataLoader
from datasets import DatasetDict
from tqdm.auto import tqdm
from einops import rearrange
import math

# MAX_CHUNK_SIZE = 1_000

# TODO: move to a separate file or something
def get_activations(f_name):
    datapoints = dataset.num_rows
    _, samp = model.run_with_cache(torch.tensor(dataset[0]["input_ids"]).to(device))
    num_features = samp[cache_name].shape[-1]
    print(f"num_features: {num_features}")
    acts_final = np.memmap(f'ACTs-{f_name}.mymemmap', dtype='float32', mode='w+', shape=(datapoints, token_amount, num_features))
    batch_size = 32

    with torch.no_grad(), dataset.formatted_as("pt"):
        dl = DataLoader(dataset["input_ids"], batch_size=batch_size)
        for i, batch in enumerate(tqdm(dl)):
            _, cache = model.run_with_cache(batch.to(device))
            batched_neuron_activations = cache[cache_name]
            next_size = min((i+1) * batch_size, datapoints)
            # print(next_size, batched_neuron_activations.shape)
            acts_final[i*batch_size:next_size,:, :] = batched_neuron_activations.view(next_size - i * batch_size, token_amount, -1).cpu().numpy()
    return acts_final

activations_main = get_activations(f"layer_{layer_focus}")

num_features: 512


  0%|          | 0/30 [00:00<?, ?it/s]

# Create the Gaussian Process Regression model

In [ ]:
!pip install gpytorch --quiet

DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)